In [1]:
import matplotlib.pyplot as plt
import numpy as np
import cupy as cp

In [2]:
from mnist import MNIST
print("Loading dataset....")
mndata = MNIST('../mnist_dataset')
X_train, y_train = mndata.load_training()
X_train = (mndata.process_images_to_numpy(X_train)/255).astype(np.float32,copy=False)
y_train = np.asarray(y_train,dtype=np.uint8)
X_test, y_test = mndata.load_testing()
X_test = (mndata.process_images_to_numpy(X_test)/255).astype(np.float32,copy=False)
y_test = np.asarray(y_test,dtype=np.uint8)
print("Done.")

Loading dataset....
Done.


In [3]:
def one_hot_encode(y):
    Yo=np.zeros((y.size,int(y.max())+1),dtype=np.float32)
    Yo[np.arange(y.size),y]=1
    return Yo

In [4]:
y_train=one_hot_encode(y_train)
y_test=one_hot_encode(y_test)

In [5]:
from nnet_gpu.network import Sequential,layers
from nnet_gpu.layers import dense
from nnet_gpu import functions
from nnet_gpu import optimizers
import numpy as np

In [6]:
X_train=cp.asarray(X_train)
y_train=cp.asarray(y_train)
X_test=cp.asarray(X_test)
y_test=cp.asarray(y_test)

In [7]:
model=Sequential()
model.add(dense(2048,input_shape=(784),activation=functions.relu))
model.add(dense(1024,activation=functions.relu))
model.add(dense(512,activation=functions.relu))
model.add(dense(256,activation=functions.relu))
model.add(dense(128,activation=functions.relu))
model.add(dense(10,activation=functions.softmax))

In [8]:
model.summary()

⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽
Layer (type)               Output Shape             Activation        Param #
- input_layer(InputLayer) (None, 784)                echo             0
__________________________________________________________________________________________
0 dense(dense)            (None, 2048)               relu             1607680
__________________________________________________________________________________________
1 dense(dense)            (None, 1024)               relu             2098176
__________________________________________________________________________________________
2 dense(dense)            (None, 512)                relu             524800
__________________________________________________________________________________________
3 dense(dense)            (None, 256)                relu             131328
_____________________________________________________________________________________

In [9]:
model.compile(optimizer=optimizers.adam,loss=functions.cross_entropy_with_logits,learning_rate=0.001)

In [ ]:
model.fit(X_train,y_train,batch_size=512,epochs=10,validation_data=(X_test,y_test))

EPOCH: 1 / 10
Progress:  60000 / 60000  - 0s - 0.041s/sample - loss: 0.0861 - accuracy: 0.9803      _
Epoch time: 5.614s
Validation Accuracy: 0.970 - val_loss: 0.4134
EPOCH: 2 / 10
Progress:  60000 / 60000  - 0s - 0.042s/sample - loss: 0.0508 - accuracy: 0.9848      _
Epoch time: 5.665s
Validation Accuracy: 0.977 - val_loss: 0.3312
EPOCH: 3 / 10
Progress:   3584 / 60000  - 5s - 0.05s/sample - loss: 0.1431 - accuracy: 0.9871      __